<a href="https://colab.research.google.com/github/DhyutiLABS/KnowYourFootPrint/blob/main/FootPrint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Carbon footprint of simple pytorch models**


**Models considered**
*   MLP [keeping the trainable parameters the same]
 - Shallow but wide [100 x 2]
 - Moderate [70 x 4]
 - Deep but narrow [40 x 8]

**Objective:** is to determine which network has a higher carbon footprint given the same order of trainable parameters.

**Methodology**

*   Use pytorch to construct the MLP models
*   Use codecarbon's `EmissionsTracker` to determine the emissions for each model built



In [ ]:
!pip install codecarbon

In [ ]:
'''
Reference for the MLP code: https://github.com/GLAZERadr/Multi-Layer-Perceptron-Pytorch/tree/main
'''

#Import all dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_csv('dataset/clean.csv')
df.head()

X = df.drop('gdp', axis=1).values
y = df['gdp'].values

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


class factbook_data:
    def __init__(self, X, y, scale_data=True):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            if scale_data:
                X = StandardScaler().fit_transform(X)
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
# MLP module
class MLP(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, hidden_neurons):
        super().__init__()
        self.layers = nn.Sequential(nn.Linear(5, hidden_neurons),
            nn.ReLU())

        for i in range(hidden_layers-1):
            self.layers.append(nn.Linear(hidden_neurons, hidden_neurons))
            self.layers.append(nn.ReLU())

        self.layers.append(nn.Linear(hidden_neurons, 1))

    def forward(self, x):
        return self.layers(x)

def model_trainer(model, dataloader, loss_function, optimizer, epochs):
    for epoch in range(epochs):
        print(f'Starting Epoch {epoch+1}')

        current_loss = 0.0

        for i, data in enumerate(trainloader, 0):
            inputs, targets = data
            inputs, targets = inputs.float(), targets.float()
            targets = targets.reshape((targets.shape[0], 1))

            optimizer.zero_grad()

            outputs = model(inputs)

            loss = loss_function(outputs, targets)

            loss.backward()

            optimizer.step()

            current_loss = loss.item()

            if i%10 == 0:
                print(f'Loss after mini-batch %5d: %.6f'%(i+1, current_loss))
                current_loss = 0.0

        print(f'Epoch {epoch+1} finished')

    print("Training has completed")

In [ ]:
dataset = factbook_data(X, y, scale_data=False)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)
testloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)

In [ ]:
next(iter(trainloader.dataset))

In [ ]:
mlp_shallow = MLP(input_size = 5, output_size = 1, hidden_layers = 8, hidden_neurons = 40 )
mlp_mid = MLP(input_size = 5, output_size = 1, hidden_layers = 4, hidden_neurons = 70 )
mlp_deep = MLP(input_size = 5, output_size = 1, hidden_layers = 8, hidden_neurons = 40 )
loss_function = nn.L1Loss()

def configure_optimizer(model):
    optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-4)
    return optimizer

In [ ]:
from codecarbon import EmissionsTracker


tracker = EmissionsTracker()
tracker.start()
try:
    # Compute intensive code goes here
    optimizer = configure_optimizer(mlp_shallow)
    model_trainer(mlp_shallow, trainloader, loss_function, optimizer, 100)

finally:
     tracker.stop()